In [3]:
from figures.hyper_opt import *

In [4]:
result = load("./optims/scenario3/forest_0_100.opt")
mdict = extract_visual(result, 10)
savemat('./figures/hyper_opt.mat', mdict=mdict, oned_as='column')

C:\Program Files\Anaconda3\envs\tensorflow\lib\site-packages\scipy\io\matlab\miobase.py:414: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  arr[empties] = ' '
